<a href="https://colab.research.google.com/github/linhtrinh213/Interpretable_VAE_TF_regulons/blob/main/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### Note
To work productively with gg colab (12 hours session):
1. Mount gg drive:
from google.colab import drive
drive.mount('/content/drive')
2. Then save or load files: model.save('/content/drive/MyDrive/my_model.h5')

- Export code (notebook) frequently (or save a copy in github)
- Save model checkpoints/variables...


To do next:
1. choose 10 real TFs
2. train 2 base models (one for the control and one for the stimulated)
(-> observe the weights)
3. retrain many times

## 1: Loading neccessities

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#### Setup ####
# install and import required packages
!pip install scanpy
!pip install decoupler
!pip install omnipath

import torch; torch.manual_seed(100)
import torch.nn as nn
import torch.utils
import torch.distributions
import torchvision
from torchvision import datasets, transforms
import math
import numpy as np
np.random.seed(100)
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
import scanpy as sc
from collections import OrderedDict
from collections import Counter

# select the right device, depending on whether your Colab runs on GPU or CPU
### IMPORTANT: we recommend to change your runtime to GPU, otherwise the training takes much longer
device = 'mps'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
    

## 2: The data

In [3]:
# Load data and use Scanpy to convert it into AnnData
PBMC_train = sc.read_h5ad("/content/drive/MyDrive/WORK/Turing Project/Interpretable_VAE/data/PBMC_train.h5ad")
regulons.to_csv('/content/drive/MyDrive/WORK/Turing Project/Interpretable_VAE/data/regulons.csv')


In [4]:
# Take a look at the data
print(PBMC_train) #the data is stored as an anndata object
print(Counter(PBMC_train.obs["cell_type"])) #summary of cell types
print(Counter(PBMC_train.obs["condition"])) #summary of conditions
# 13515 observation: cells (.obs attribute)
# gene expression data: .X (sparse matrix)
# annotations for the variables (genes) in the .var attribute
# the attribute have specific features that can be accessed using squared bracktets

AnnData object with n_obs × n_vars = 13515 × 6998
    obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type'
    var: 'gene_symbol', 'n_cells'
    uns: 'cell_type_colors', 'condition_colors', 'neighbors'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'
Counter({'CD4T': 4452, 'FCGR3A+Mono': 2881, 'CD14+Mono': 2049, 'B': 1448, 'NK': 931, 'CD8T': 892, 'Dendritic': 862})
Counter({'stimulated': 7109, 'control': 6406})


In [5]:
# devide data into control and stimulated

# Subset for a specific condition, e.g., "control"
PBMC_control = PBMC_train[PBMC_train.obs["condition"] == "control"].copy()

# Another example for "treated"
PBMC_stimulated = PBMC_train[PBMC_train.obs["condition"] == "stimulated"].copy()


In [6]:
# Define Encoder:
class Encoder(nn.Module):
    def __init__(self, latent_dims, input_dims, dropout, z_dropout): #dropout between the dense layers, z_dropout define the dropout rates between the encoder/latent space
        super(Encoder, self).__init__() #run the initialize code from nn.Module -> this class behaves like a Pytorch model
        self.encoder = nn.Sequential(
                                     nn.Linear(input_dims, 800),
                                     nn.ReLU(),
                                     nn.Dropout(p = dropout),
                                     nn.Linear(800, 800),
                                     nn.ReLU(),
                                     nn.Dropout(p = dropout))  #two layer, fully connected encoder with dropout

        # outputs mean vector u
        self.mu = nn.Sequential(nn.Linear(800, latent_dims), # the 800 neurons in the second layers -> latent space
                                nn.Dropout(p = z_dropout))
        # outputs standard variance
        self.sigma = nn.Sequential(nn.Linear(800, latent_dims),
                                   nn.Dropout(p = z_dropout))

        self.N = torch.distributions.Normal(0, 1)  # define Gaussian distribution for each input
        self.N.loc = self.N.loc.to(device) # move to the right device
        self.N.scale = self.N.scale.to(device)
        self.kl = 0 # place holder for storing KL divergence (regularization term)
        # KL measures how far the learned Gaussian is from the standard normal (0,1) -> this is a regularization term in VAE
    def forward(self, x):
        x = self.encoder(x) # pass the data to the encoder
        mu =  self.mu(x) # predict mean vector
        sigma = torch.exp(self.sigma(x)) # predict standard var exp for numeric stability
        z = mu + sigma*self.N.sample(mu.shape)  # Sample z using reparameterization trick

        self.kl = (0.5*sigma**2 + 0.5*mu**2 - torch.log(sigma) - 1/2).sum() #calculation of kullback-leibler divergence

        return z # output is the sampled latent vector


## 3: regulons and create the mask

To achieve interpretability through model's decoder:
1. one-layer decoder: every latent var is directly connected to the output var
2. linear decoder: no activation function in the decoder
3. positive decoder: restrict the decoder weights to positive values (all negative weights = 0) (*why not - weight?, cant it be for inhibition?)
=> for this project: TF: It can be negative!
4. sparse decoder: latent node = TF, which only connect to certain genes


- How do we do that?
++ mask M: define decoder connections. Rows = genes. Columns = TF. If TF j has an effect on gene i -> the corresponding entry in the matrix M(i,j) contains the value 1. ??: has an effect = activate AND inhibit, or only activate?
So mask M: only defines which latent node **connect** to which decoder node, and the weights will learn + or - (activation or inhibition, or at least thats what we expect).
++ mask M: also includes fully connected nodes (all values = 1) -> for better data reconstruction and compensates for connections that are missing

In [7]:
# processed regulons
import decoupler as dc
regulons = dc.get_collectri(organism='human', split_complexes=False)
# CollecTRI netork: a curated collection of TFs and their transcriptional targets
# spilt_complexes: keep or spilt complexes into subunits
# weight: 1 is activation. -1 is inhibition
# raw regulons
#import omnipath as op
#op.interactions.CollecTRI.get(genesymbols=True, organism=9606L, loops=True)#

In [28]:
regulons.target.unique()

<StringArray>
[    'BAX',    'BCL2',    'BCL6',   'CCND2',  'CDKN1A',    'CSF1',   'FOXO3',
     'JUN',    'PIM1',    'TP53',
 ...
  'RNF114',   'ETNK1',    'CHL1',    'DSC1', 'GPRASP1',   'MAGI2',  'REC114',
   'SYNJ1',   'TCP11', 'PIK3AP1']
Length: 6627, dtype: string

In [11]:
import pandas as pd
import numpy as np

def create_mask(adata, regulons, add_nodes:int=10, sep = "\t"):
    """
    Initialize mask M that specifies which latent nodes connect to which decoder nodes.
    Args:
        adata (Anndata): Scanpy single-cell object, we will store the computed mask and the names of the biological processes there
        regulons: which TFs affected which genes
        add_nodes (int): Additional latent nodes for capturing additional variance
    Return:
        adata (Anndata): Scanpy single-cell object that now stores the computed mask and the names of biological processes (in the .uns["_vega"] attribute)
        mask (array): mask M that specifies whether a gene is included in the gene set of a pathway (value one) or not (value zero)
    """

    # Create the mask
    # 1. Get unique genes (targets) and TFs (sources)
    genes = regulons['target'].unique()
    tfs = regulons['source'].unique()
    # randomly choose 10 distinct TF‐indices
    selected_idx = np.random.choice(len(tfs), size=10, replace=False)

    # get their names and the corresponding sub‐mask
    selected_tfs  = [tfs[i]   for i in selected_idx]

    # 2. Initialize matrix M with zeros
    M = pd.DataFrame(0, index=genes, columns=tfs)

    # 3. Set M[i,j] = 1 where the gene i is affected by TF j
    for _, row in regulons.iterrows(): # for each row in regulons
        M.loc[row['target'], row['source']] = 1 #the corresponding genes, TF box = 1

    M = M.iloc[:, selected_idx]  # if M is a pandas DataFrame

    # Add unannotated nodes
    vec = np.ones((M.shape[0], add_nodes))
    M = np.hstack((M, vec))

    adata.uns['_vega'] = dict() #create attribute "_vega" to store the mask and pathway information
    adata.uns['_vega']['mask'] = M
    adata.uns['_vega']['TFs'] = list(tfs) + ['UNANNOTATED_'+str(k) for k in range(add_nodes)]

    return adata, M

In [12]:
# apply the create_mask function
PBMC_control, mask_ctr = create_mask(PBMC_control,regulons , add_nodes=1)
PBMC_stimulated, mask_sti  = create_mask(PBMC_stimulated,regulons , add_nodes=1)

In [15]:
print(mask_ctr)

[[0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


## 4: Decoder


In [34]:
# filter the genes
genes_mask = np.array(regulons.target.unique()) # genes in regulons list

pbmc_genes = np.array(PBMC_train.var_names)# genes in OUR data

# Create boolean mask of which genes are in PBMC_train
keep = np.isin(genes_mask, pbmc_genes)

# Apply the filter
filtered_mask_ctr = mask_ctr[keep, :]
filtered_mask_sti = mask_sti[keep, :]

In [33]:
filtered_mask_ctr.shape #2k genes, 11 (10 + 1)

(2168, 11)

In [55]:
gene_keep = genes_mask[keep]
PBMC_control_filtered = PBMC_control[:, PBMC_control.var_names.isin(gene_keep)].copy()


In [56]:
PBMC_control_filtered # 2k genes

AnnData object with n_obs × n_vars = 6406 × 2168
    obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type'
    var: 'gene_symbol', 'n_cells'
    uns: 'cell_type_colors', 'condition_colors', 'neighbors'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [35]:
# define VEGA's decoder

class DecoderVEGA(nn.Module):
  """
  Define VEGA's decoder (sparse, one-layer, linear, positive)
  """
  def __init__(self,
               mask):
        super(DecoderVEGA, self).__init__()

        self.sparse_layer = nn.Sequential(SparseLayer(mask)) # we define the architecture of the decoder below with the class "SparseLayer"

  def forward(self, x):
    z = self.sparse_layer(x.to(device))
    return(z)

# define a class SparseLayer, that specifies the decoder architecture (sparse connections based on the mask)
class SparseLayer(nn.Module):
  def __init__(self, mask):
        """
        Extended torch.nn module which mask connection
        """
        super(SparseLayer, self).__init__()

        self.mask = nn.Parameter(torch.tensor(mask, dtype=torch.float).t(), requires_grad=False)
        self.weight = nn.Parameter(torch.Tensor(mask.shape[1], mask.shape[0]))
        self.bias = nn.Parameter(torch.Tensor(mask.shape[1]))
        self.reset_parameters()

        # mask weight
        self.weight.data = self.weight.data * self.mask

  def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

  def forward(self, input):
        # See the autograd section for explanation of what happens here
        return SparseLayerFunction.apply(input, self.weight, self.bias, self.mask)


######### You don't need to understand this part of the code in detail #########
class SparseLayerFunction(torch.autograd.Function):
    """
    We define our own autograd function which masks it's weights by 'mask'.
    For more details, see https://pytorch.org/docs/stable/notes/extending.html
    """

    # Note that both forward and backward are @staticmethods
    @staticmethod
    def forward(ctx, input, weight, bias, mask):

        weight = weight * mask # change weight to 0 where mask == 0
        #calculate the output
        output = input.mm(weight.t())
        output += bias.unsqueeze(0).expand_as(output) # Add bias to all values in output
        ctx.save_for_backward(input, weight, bias, mask)
        return output

    @staticmethod
    def backward(ctx, grad_output): # define the gradient formula
        input, weight, bias, mask = ctx.saved_tensors
        grad_input = grad_weight = grad_bias = grad_mask = None

        # These needs_input_grad checks are optional and only to improve efficiency
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
            # change grad_weight to 0 where mask == 0
            grad_weight = grad_weight * mask
        if ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias, grad_mask


## 4: Combine encoder and decoder

In [62]:
# define class that combine encoder and decoder
class VEGA(nn.Module):
    def __init__(self, latent_dims, input_dims, mask, dropout = 0.3, z_dropout = 0.3):
        super(VEGA, self).__init__()
        self.encoder = Encoder(latent_dims, input_dims, dropout, z_dropout) # we use the same encoder as before (two-layer, fully connected, non-linear)
        self.decoder = DecoderVEGA(mask)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [61]:
#training loop
def trainVEGA(vae, data, epochs=50, beta = 0.0001, learning_rate = 0.01):
    opt = torch.optim.Adam(vae.parameters(), lr = learning_rate, weight_decay = 5e-4)
    vae.train() #train mode
    losses = []
    klds = []
    mses = []

    for epoch in range(epochs):
        loss_e = 0
        kld_e = 0
        mse_e = 0

        for x in data:
            x = x.to(device)
            opt.zero_grad()
            x_hat = vae(x)
            mse = ((x - x_hat)**2).sum()
            kld = beta* vae.encoder.kl
            loss = mse +  kld # loss calculation
            loss.backward()
            opt.step()
            loss_e += loss.to('cpu').detach().numpy()
            kld_e += kld.to('cpu').detach().numpy()
            mse_e += mse.to('cpu').detach().numpy()

        losses.append(loss_e/(len(data)*128))
        klds.append(kld_e/(len(data)*128))
        mses.append(mse_e/(len(data)*128))

        print("epoch: ", epoch, " loss: ", loss_e/(len(data)*128))

    return vae, losses, klds, mses

In [57]:
# train the model for control
PBMC_controlX = torch.utils.data.DataLoader(PBMC_control_filtered.X.toarray(), batch_size=128) #set up the training data in the right format
PBMC_stimulatedX = torch.utils.data.DataLoader(PBMC_stimulated.X.toarray(), batch_size=128) #set up the training data in the right format



In [41]:
import torch

# NOTE: In Runtime, Change type (hardware accelerator) -> GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [67]:
# train the model for control
vega_ctr = VEGA(latent_dims= filtered_mask_ctr.shape[1], input_dims = filtered_mask_ctr.shape[0], mask = filtered_mask_ctr.T, z_dropout = 0.1, dropout = 0.1).to(device) # input_dim should be the

# latent_dims should be number of TFs
# input dims = PBMC_train.shape[1]??
# model training
vega_ctr, vega_losses_ctr, vega_klds_ctr, vega_mses_ctr = trainVEGA(vega_ctr, PBMC_controlX,epochs = 50, beta = 0.00001) #takes about 2 mins on GPU # change beta!!!
# gotta change dropout rate as well otherwise its too instable

epoch:  0  loss:  429.96216
epoch:  1  loss:  159.67598
epoch:  2  loss:  136.07414
epoch:  3  loss:  123.63394
epoch:  4  loss:  117.009125
epoch:  5  loss:  113.12189
epoch:  6  loss:  113.404816
epoch:  7  loss:  109.70375
epoch:  8  loss:  108.13356
epoch:  9  loss:  106.626976
epoch:  10  loss:  103.705635
epoch:  11  loss:  101.3185
epoch:  12  loss:  98.31986
epoch:  13  loss:  96.75497
epoch:  14  loss:  96.3989
epoch:  15  loss:  96.61102
epoch:  16  loss:  94.474045
epoch:  17  loss:  94.14765
epoch:  18  loss:  93.84986
epoch:  19  loss:  93.33594
epoch:  20  loss:  92.97582
epoch:  21  loss:  93.097084
epoch:  22  loss:  92.63841
epoch:  23  loss:  92.68052
epoch:  24  loss:  92.26901
epoch:  25  loss:  92.31285
epoch:  26  loss:  92.646324
epoch:  27  loss:  92.810776
epoch:  28  loss:  93.55039
epoch:  29  loss:  92.31476
epoch:  30  loss:  92.50665
epoch:  31  loss:  93.0506
epoch:  32  loss:  89.37131
epoch:  33  loss:  89.00345
epoch:  34  loss:  88.98653
epoch:  35  l

Error: mat1xmat2 dimensions (mask have 6k7 genes. our data have 6k9 genes -> at some point they will not be compatible)
-> How to fix it:

1. For the mask: which gene in the in the regulon list is in OUR data (for now: around 2k genes)
Only keep those 2k genes for the PBMC as well !!

2. Plus, choose only around 10 TFs so that training faster.